In [3]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')   # optional but recommended for better lemmatization


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LAXMIKANT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\LAXMIKANT\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
df = pd.read_csv("tweet_emotions.csv")
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [5]:
df.drop(columns=['tweet_id'], inplace=True)

In [6]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\LAXMIKANT\AppData\Local\Temp\ipykernel_21320\93639238.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [7]:
import nltk
nltk.download('wordnet')
df = normalize_text(df)
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LAXMIKANT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [8]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [9]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [10]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

C:\Users\LAXMIKANT\AppData\Local\Temp\ipykernel_21320\1089524538.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want he s mar...
8,0,charviray charlene love miss
9,0,kelcouch i m sorry least friday


In [11]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']



In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/laxmikantbabaleshwar07/mlops_mini_project.mlflow')
dagshub.init(repo_owner='laxmikant', repo_name='mlops-mini-project', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\ML\Project ML\mlops_mini_project\myenv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=80b982f3-c45c-463c-981c-0a578b06877f&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=c2981ff5f60bf7fa6ce1961438d41e3fbcbdc731c0671e61258024561ef8882a




Accessing as laxmikantbabaleshwar07

Repository mlops-mini-project doesn't exist, creating it under current user.

Initialized MLflow to track repo "laxmikant/mlops-mini-project"

Repository laxmikant/mlops-mini-project initialized!

2025/10/02 10:06:58 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/fc75a4e1848a435ebda62734f5ea12df', creation_time=1759379819093, experiment_id='0', last_update_time=1759379819093, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [18]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle
import os

with mlflow.start_run():
    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)
    
    # Model training
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")
    
    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # 🚀 Instead of log_model → save and log manually as artifact
    model_path = "model.pkl"
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    mlflow.log_artifact(model_path)   # ✅ Works on DagsHub

    # Log the notebook
    notebook_path = "exp1_baseline_model.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")


Accuracy: 0.7788
Precision: 0.7715
Recall: 0.7783
F1 Score: 0.7749
🏃 View run mercurial-toad-995 at: https://dagshub.com/laxmikantbabaleshwar07/mlops_mini_project.mlflow/#/experiments/0/runs/dbf4101f366d40c3ad3244dda64fdb8b
🧪 View experiment at: https://dagshub.com/laxmikantbabaleshwar07/mlops_mini_project.mlflow/#/experiments/0
